In [1]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None)

In [7]:
df = pd.read_csv("utils/fifa_data_without_duplicates")

In [8]:
df.head()

,name,age,overall_rating,height,weight,potential,preferred_foot,best_overall,best_postion,market_value,attacking,team,contract_duration,joined,id,gk_diving,gk_handling,gk_kicking,gk_positioning,gk_reflexes,week_foot,skill_moves,pace,shooting,passing,dribbling,defending,physics,total_small_stats
0,J. Cavallo,21,59,176cm,71kg,66,Left,60,CAM,€425K,229,Adelaide United,\r\n2021 ~ 2023,"Feb 23, 2021",245509,15,10,14,13,10,3 ★,2★,70,43,55,60,53,66,1591
1,14 Adriano,31,73,189cm,88kg,73,Left,73,ST,€1.5M,338,Club Athletico Paranaense,\r\n2014 ~ 2014,"Feb 11, 2014",106019,13,6,8,15,14,2 ★,3★,40,78,58,67,32,65,1629
2,R. Perraud,23,77,173cm,68kg,83,Left,78,LWB,€15.5M,351,Southampton,\r\n2021 ~ 2025,"Jul 2, 2021",231318,6,7,10,11,13,2 ★,3★,74,72,75,74,72,73,2092
3,J. Rodríguez,29,81,180cm,75kg,81,Left,83,CAM,€25.5M,409,Everton,\r\n2020 ~ 2022,"Sep 7, 2020",198710,15,15,15,5,14,3 ★,4★,53,86,86,86,50,60,2091
4,M. Cash,23,78,179cm,74kg,83,Right,78,RB,€20M,311,Aston Villa,\r\n2020 ~ 2025,"Sep 3, 2020",227174,15,16,13,13,8,3 ★,3★,81,63,68,73,72,66,1971


# General Cleaning

In [9]:
# Converts to the correct the input value to the correct number (eg. 10k --> 10,000, 1M --> 1,000,000)
def convert_to_number(value):
    value = value[1:]
    if value[-1] == "K":
        return float(value.replace("K", "")) * 1000
    elif value[-1] == "M":
        return float(value.replace("M", "")) * 1000000
    else:
        return float(value)    

In [10]:
# Data Cleaning --> Remove unnecessary characters
df["height"] = df["height"].str[:-2].astype(np.int64)
df["weight"] = df["weight"].str[:-2].astype(np.int64)
df['market_value'] = df['market_value'].apply(convert_to_number)
df["weak_foot"] = df["week_foot"].str[0]
df["skill_moves"] = df["skill_moves"].str[0]

# Delete players that have market value of zero
df.drop(df[df['market_value'] == 0].index, inplace = True, axis = 0)

# Indicate loan players
df["loan_player"] = np.where(df["contract_duration"].str.contains("loan", case = False), True, False)

# Get contract_end and drop players which have contract end older than 2020 (so called icons in fifa)
df["contract_end"] = df["contract_duration"].str[8:]
df["contract_end"] = df["contract_end"].str.extract('(\d+)').astype(np.float64)
df.drop(df[df['contract_end'] < 2020].index, inplace = True)

# Get contract start, for loan players this data is missing. For now we set the contract start of loan players to missing
df["contract_start"] = np.where(df["loan_player"] == True, np.nan, df["contract_duration"].str[2:6]).astype(np.float64)

#Delete unnecassary columns
df.drop(["contract_duration", "joined"], axis = 1, inplace = True)


# Missing Value Analysis

There are only missing value for the column contract_start. These are loan player and for these players the table of sofifa did not provide the contract start. Therefore we have to impute this data because we want to keep the loan players in the data. We will impute the data by the median. 

In [6]:
df.isnull().sum().sort_values(ascending=False).head(30)

contract_start       1173
gk_handling             0
age                     0
overall_rating          0
height                  0
weight                  0
potential               0
preferred_foot          0
best_overall            0
best_postion            0
market_value            0
attacking               0
team                    0
id                      0
gk_diving               0
gk_kicking              0
contract_end            0
gk_positioning          0
gk_reflexes             0
week_foot               0
skill_moves             0
pace                    0
shooting                0
passing                 0
dribbling               0
defending               0
physics                 0
total_small_stats       0
weak_foot               0
loan_player             0
dtype: int64

In [222]:
#df["contract_start"].fillna(df["contract_start"].median(), inplace = True)
df["contract_start"] = df[["contract_end", "contract_start"]].groupby("contract_end").transform(lambda x: x.fillna(x.median()))

In [204]:
df.to_csv("utils/fifa_data_cleaned", encoding='utf-8', index=False)